<a href="https://colab.research.google.com/github/ShinAsakawa/ShinAsakawa.github.io/blob/master/2022notebooks/2022_0904exercise_mecab_etc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

* filename: `2022_0904exercise_around_mecab.ipynb`
* author: 浅川伸一

* MeCab と松下データを使って任意の文章を，分かち書きして，各品詞の松下データの頻度を検索して，その頻度最大値を返す。
2022_0821 の zoom ミーティングからの宿題に対する回答コード

* サンプルデータとして，京都大学黒橋研究室作成の SNLI データセットを用いることとした

In [1]:
# Mac の retina ディスプレイの場合，高解像度の画面を使用する
%config InlineBackend.figure_format = 'retina'

# 下準備，実行環境諸元の表示と設定
try:
    import bit
except ImportError:
    !pip install ipynbname
    !git clone https://github.com/ShinAsakawa/bit.git
import bit
isColab = bit.isColab
HOME = bit.HOME


日付: 2022-09-18
HOSTNAME: 3f41b67d0103
ユーザ名: root
HOME: /root
ファイル名: /fileId=https%3A%2F%2Fgithub.com%2FShinAsakawa%2FShinAsakawa.github.io%2Fblob%2Fmaster%2F2022notebooks%2F2022_0904exercise_mecab_etc.ipynb
torch.__version__: 1.12.1+cu113


## colab 上で必要なライブラリのインストールなど

In [ ]:
# if isColab:
#     !apt install aptitude
#     !aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
#     !pip install mecab-python3==0.7

In [2]:
import os
import sys
import numpy as np
import random

# ローカルと colab との相違を吸収するために
# 本ファイルを Google Colaboratory 上で実行する場合に，必要となるライブラリをインストール
if isColab:
    !git clone https://github.com/ShinAsakawa/ccap.git
    !pip install japanize_matplotlib > /dev/null 2>&1
    !pip install jaconv > /dev/null 2>&1

    # MeCab, fugashi, ipadic のインストール
    !apt install aptitude swig > /dev/null 2>&1
    !aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y > /dev/null 2>&1
    !pip install mecab-python3==0.7 > /dev/null 2>&1
    !pip install --upgrade ipadic > /dev/null 2>&1

    #!git clone --depth 1 https://github.com/neologd/mecab-ipadic-neologd.git > /dev/null 2>&1
    #!echo yes | mecab-ipadic-neologd/bin/install-mecab-ipadic-neologd -n -a > /dev/null 2>&1
    
    #import subprocess
    #cmd='echo `mecab-config --dicdir`\"/mecab-ipadic-neologd\"'
    #path_neologd = (subprocess.Popen(cmd, stdout=subprocess.PIPE,
    #                                 shell=True).communicate()[0]).decode('utf-8')

    #!pip install 'konoha[mecab]'
    #!pip install 'fugashi[unidic]' > /dev/null 2>&1
    #!python -m unidic download > /dev/null 2>&1
    !pip install jaconv
    #!pip install transformers fugashi ipadic    

fatal: destination path 'ccap' already exists and is not an empty directory.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## インストール補足

In [3]:
import IPython
isColab = 'google.colab' in str(IPython.get_ipython())
if isColab:
    !pip install --upgrade openpyxl    # エクセルファイルを読むこむ際にバージョンの相違で動作しない場合があるので念の為
    !pip install --upgrade pandas      # 同上
    #!python -m unidic download
    #!pip install --upgrade fugashi[unidic-lite]
    #!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


# 日本語を読むための”ＴＭ語彙リスト”（総合版）　Ver.4.0 の読み込み

In [4]:
#東大の松下研究室の辞書を読み込む
import requests
import pandas as pd
import os
import jaconv

# 日本語を読むための語彙データベース（VDRJ） Ver. 1.1　（＝日本語を読むための”ＴＭ語彙リスト”（総合版）　Ver.4.0）
vdrj_url='http://www17408ui.sakura.ne.jp/tatsum/database/VDRJ_Ver1_1_Research_Top60894.xlsx'

# 直上行の url からエクセルファイル名を切り出す
excel_fname = vdrj_url.split('/')[-1]

# もしエクセルファイルが存在しなかったら ダウンロードする
if not os.path.exists(excel_fname):       
    r = requests.get(vdrj_url)
    with open(excel_fname, 'wb') as f:
        total_length = int(r.headers.get('content-length'))
        print('Downloading {0} - {1} bytes'.format(excel_fname, (total_length)))
        f.write(r.content)
        
# 松下データの読み込み，シート名を指定
sheet_name='重要度順語彙リスト60894語'

# 実際のエクセルファイルの読み込み
vdrj_df = pd.read_excel(excel_fname, sheet_name=sheet_name)

## 読み込んだ語彙リストの変換

In [5]:
import unicodedata
# `日本語を読むための”ＴＭ語彙リスト”（総合版）　Ver.4.0` の列名データ
cols = ['留学生用\n語彙レベル\nWord Level for International Students',
        '留学生用語彙ランク\nWord Ranking for International Students',
        '一般語彙レベル\nWord Level for General Learners',
        '一般語彙ランク\nWord Ranking for General Learners',
        '書きことば語彙レベル\nWord Level for Written Japanese',
        '書きことば重要度ランク（想定既知語彙を除く）\nU Ranking for Written Japanese excluding Assumed Known Words',
       '旧日本語能力試験出題基準レベル\nOld JLPT Level',
        '語彙階層ラベル Word Tier Label\n\n学術共通語彙：4Dまたは3Dを含む語\n（旧日能試4級語彙および留学生用語彙レベル21K+の語彙を除く）\n\n限定学術領域語彙：2Dまたは1Dを含む語（旧日能試4級語彙および留学生用語彙レベル21K+の語彙を除く）',
        '見出し語彙素\nLexeme', 
        '標準的（新聞）表記\nStandard (Newspaper) Orthography',
        '標準的読み方（カタカナ）\nStandard Reading (Katakana)', 
        '品詞\nPart of Speech',
        '使用度数\nFrequency',
        '修正済み使用度数（総延べ語数32656221語中）\nCorrected Frequency (Out of Total Token 32656221)',
        '修正度数\nFrequency for Correction ',
        '10分野100万語あたり使用頻度(Fw)\nStandardized Freq/\nmillion in 10 Written Domains (Fw)',
        '(Fw)累積テキストカバー率（想定既知語彙分を含む）\nFw Cumulative Text Coverage including Assumed Known Words',
        '\n書字形（例）\nOrthographic Form Example',
        '発音形（例）\nPhonological Form Example', '語彙素読み\nReading of Lexeme',
        '活用型\nConjugation Type', '活用形\nConjugated Form Example',
        '語形\nWord Form', 'ID',
        'ホームポジション並べ替え用ID\nID for Sorting by the Original Order']

x = vdrj_df[cols]

# 列名が長くてダルいので簡易な列名に変換しておく
vdrj_df_ = x.rename(columns = {
    '留学生用\n語彙レベル\nWord Level for International Students': 'wlevel_int',
    '留学生用語彙ランク\nWord Ranking for International Students': 'wrank_int',
    '一般語彙レベル\nWord Level for General Learners'           : 'wlevel_gen',
    '一般語彙ランク\nWord Ranking for General Learners'         : 'wrand_gen',
    '書きことば語彙レベル\nWord Level for Written Japanese'      : 'wrank_gen',
    '書きことば重要度ランク（想定既知語彙を除く）\nU Ranking for Written Japanese excluding Assumed Known Words': 'Urank',
    '旧日本語能力試験出題基準レベル\nOld JLPT Level': 'jlpt_level',
    '語彙階層ラベル Word Tier Label\n\n学術共通語彙：4Dまたは3Dを含む語\n（旧日能試4級語彙および留学生用語彙レベル21K+の語彙を除く）\n\n限定学術領域語彙：2Dまたは1Dを含む語（旧日能試4級語彙および留学生用語彙レベル21K+の語彙を除く）': 'word_tier_label',
    '見出し語彙素\nLexeme': 'lexeme', 
    '標準的（新聞）表記\nStandard (Newspaper) Orthography': 'newspaper',
    '標準的読み方（カタカナ）\nStandard Reading (Katakana)': 'katakana',
    '品詞\nPart of Speech'                              : 'pos',
    '使用度数\nFrequency'                                : 'freq',
    '修正済み使用度数（総延べ語数32656221語中）\nCorrected Frequency (Out of Total Token 32656221)': 'freq_correct',
    '修正度数\nFrequency for Correction '                : 'freq_correct2',
    '10分野100万語あたり使用頻度(Fw)\nStandardized Freq/\nmillion in 10 Written Domains (Fw)'    : 'freq_std',
    '(Fw)累積テキストカバー率（想定既知語彙分を含む）\nFw Cumulative Text Coverage including Assumed Known Words': 'cum_freq',
    '\n書字形（例）\nOrthographic Form Example'           : 'orth_example',
    '発音形（例）\nPhonological Form Example'             : 'phon_example', 
    '語彙素読み\nReading of Lexeme'                       : 'lexeme_reading',
    '活用型\nConjugation Type' : 'conj_type', 
    '活用形\nConjugated Form Example': 'conj_example',
    '語形\nWord Form': 'word_form', 
    'ID': 'ID',
    'ホームポジション並べ替え用ID\nID for Sorting by the Original Order': 'ID_sort'})


def jaconv_normalize(word:str, mode='NFKC'):
    """松下データの 'Lexeme' と '語形' には文字列ではないデータや空欄がある。
    そのため，例外処理をすることにした
    """
    if not isinstance(word, str):
        return str(word)
    else:
        return unicodedata.normalize(mode, word)
        #return jaconv.normalize(word)

# 上で定義した関数 `jaconv_normalize()` を使って松下データ vdrj の文字を NFKC に正規化
# おそらく松下データは windows で作成された NFD であると思われるので，念の為
vdrj_lexeme    = [jaconv_normalize(w) for w in vdrj_df_['lexeme'].to_list()]
vdrj_word_form = [jaconv_normalize(w) for w in vdrj_df_['word_form'].to_list()]
vdrj_freq      = [jaconv_normalize(w) for w in vdrj_df_['freq']]
vdrj_ids = [int(w) for w in vdrj_df_['ID'].to_list()]

#確認用 すべて同じ数 60894 になっているはず
for x in [vdrj_lexeme, vdrj_word_form, vdrj_freq, vdrj_ids]:
    print(len(x))

# データのチェック
# 松下データには重複があるらしい
word_count = {}
for i, w in enumerate(vdrj_lexeme):
    if not w in word_count:
        word_count[w] = 1
    else:
        word_count[w] += 1
        print(f'{i:5d} 単語: {w} 重複')
for k, v in word_count.items():
    if word_count[k] != 1:
        print(f'単語: {k}, 重複回数: {v}')
        

vdrj = {}          # ここで用いる松下データの本体
vdrj_wrd2idx = {}  # 任意の単語を松下 ID に変換するための辞書
for i, (lexeme, word_form, freq, idx) in enumerate(zip(vdrj_lexeme, vdrj_word_form, vdrj_freq, vdrj_ids)):
    vdrj[idx] = {'lexeme': lexeme,
                 'word_form': word_form,
                 'freq': int(freq),
                }
    vdrj_wrd2idx[lexeme] = idx
#list(vdrj.keys())[3:10]
#vdrj_wrd2idx['突然']

60894
60894
60894
60894
59925 単語: ping 重複
60600 単語: 突然 重複
単語: 突然, 重複回数: 2
単語: ping, 重複回数: 2


# 日本語版 NLI データセットの読み込み

In [ ]:
from zipfile import ZipFile

jsnli_url = 'https://nlp.ist.i.kyoto-u.ac.jp/DLcounter/lime.cgi?down=https://nlp.ist.i.kyoto-u.ac.jp/nl-resource/JSNLI/jsnli_1.1.zip&name=JSNLI.zip'
zip_fname = jsnli_url.split('/')[-1].split('&')[0]
#zip_fname = jsnli_zip_fname
if not os.path.exists(zip_fname):  # もしエクセルファイルが存在しなかったら ダウンロードする
    r = requests.get(jsnli_url)
    with open(zip_fname, 'wb') as f:
        total_length = int(r.headers.get('content-length'))
        print('Downloading {0} - {1} bytes'.format(zip_fname, (total_length)))
        f.write(r.content)

def extract_zip(input_zip):
    input_zip=ZipFile(input_zip)
    return {name: input_zip.read(name) for name in input_zip.namelist()}        

a = extract_zip(zip_fname)
jsnli_train_wo = a['jsnli_1.1/train_wo_filtering.tsv'].decode('utf-8').split('\n')
jsnli_train_w  = a['jsnli_1.1/train_w_filtering.tsv'].decode('utf-8').split('\n')
jsnli_dev      = a['jsnli_1.1/dev.tsv'].decode('utf-8').split('\n')

def jsnli_strip_sentenses(data:list):
    ret = []
    for line in data:
        x = []
        for p in line.split('\t'):
            x.append(p.replace(' ',''))
        ret.append(x)
    return ret

JSNLI = {}
for k,v in {'train_wo':jsnli_train_wo, 'train_w':jsnli_train_w, 'dev':jsnli_dev}.items():
    JSNLI[k] = jsnli_strip_sentenses(v)

# JSNLI データセットのサイズを表示    
for k, v in JSNLI.items():
    print(f'データセット名:{k}, サイズ:{len(v)}')
    

In [ ]:
p  = tagger.parse('本日は悪天なり。').strip().split('\n')
for _p in p[:-1]:
    print(_p, _p.split(','), _p.split(',')[-3])

In [ ]:
import numpy as np
from termcolor import colored
import MeCab

tagger = MeCab.Tagger()

def mecab_get_origin(sent:str):
    """MeCab を使って，文を分解し，原形と品詞からなるリストを返す"""
    p = tagger.parse(sent).strip().split('\n')
    ret = []
    for _p in p[:-1]:
        # 分解された各要素の 4 番目を取り出す
        x = _p.split(',')
        orig = x[-3]
        orig = orig.split('-')[0] if '-' in orig else orig
        pos = x[0]
        ret.append((orig, pos))
    return ret
    
def get_vdrj_wrd2idx(wrd:str,
                 wrd2idx:dict=vdrj_wrd2idx,
                ):
    if wrd in wrd2idx:
        return wrd2idx[wrd]
    else:
        return -1

In [ ]:
sent = '明日天気なーれ。'

# 上で定義した sent を使って，MeCab を用いて，単語の原形を取り出す
print(f'文の品詞分解と原形: {mecab_get_origin(sent)}')

# 取り出した単語の原形のリストから松下データの　ID を取り出す
print(f'各単語の ID リスト: {[get_vdrj_wrd2idx(x[0]) for x in mecab_get_origin(sent)]}')

In [ ]:
for d in JSNLI['train_wo'][:10]:
    _, s1, s2 = d
    for s in [s1, s2]:
        mecab_parsed = mecab_get_origin(s)
        ids = [get_vdrj_wrd2idx(x[0]) for x in mecab_parsed]
        print(colored(f'最大松下ID:{np.max(ids)}', 'blue', attrs=['bold']),
              f'原文:{s}', 
              f'ID リスト:{ids}')